In [8]:
from transformers import BlipProcessor, BlipForConditionalGeneration
import torch

In [10]:
if torch.cuda.is_available():
    device = torch.device("cuda")
elif torch.backends.mps.is_available():
    device = torch.device("mps")   # Apple GPU
else:
    device = torch.device("cpu")

print(f"Using device: {device}")

Using device: mps


In [ ]:
# Load BLIP model
processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-large")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-large").to(device)
model.eval()
print("BLIP model loaded")

BLIP model loaded (LARGE version like paper)


In [1]:
#Hallo2